In [1]:
!pip install pennylane

In [2]:
import pennylane as qml
from pennylane.operation import Operation
import numpy as np
import tensorflow as tf

## RBS GATE

In [3]:
class RBSGate(Operation):
    num_params = 1
    num_wires = 2
    par_domain = 'R'

    def __init__(self, theta, wires):
        super().__init__(theta, wires=wires)
        self.theta = theta

    @staticmethod
    def compute_matrix(theta):
        cos = np.cos(theta)
        sin = np.sin(theta)
        return np.array([
            [1, 0, 0, 0],
            [0, cos, sin, 0],
            [0, -sin, cos, 0],
            [0, 0, 0, 1]
        ])

    def adjoint(self):
        return RBSGate(-self.parameters[0], wires=self.wires)

    def label(self, decimals=None, base_label=None, **kwargs):
        theta = self.parameters[0]
        return f"RBS({theta:.2f})"

# Test the custom gate
theta = np.pi / 4

# Step 2: Create a quantum device
dev = qml.device('default.qubit', wires=2)

# Step 3: Construct the circuit using the custom gate
@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    RBSGate(theta, wires=[0, 1])
    return qml.state()

In [4]:
# Execute the circuit and print the results
state = circuit()
print("State vector:")
print(state)

# Draw the circuit
print("Circuit diagram:")
print(qml.draw(circuit)())

State vector:
[0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]
Circuit diagram:
0: ──X─╭RBS(0.79)─┤  State
1: ────╰RBS(0.79)─┤  State


In [5]:
# Extracting and printing the unitary matrix
@qml.qnode(dev)
def unitary_circuit():
    RBSGate(theta, wires=[0, 1])
    return qml.state()

unitary_matrix = qml.matrix(unitary_circuit)()
print("Unitary matrix from Pennylane:")
print(unitary_matrix)


Unitary matrix from Pennylane:
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.         -0.70710678  0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]


## Vector Loader

In [6]:
def convert_array(X):
    alphas = tf.zeros(X.shape[:-1] + (X.shape[-1]-1,), dtype=X.dtype)
    X_normd =  tf.linalg.l2_normalize(X, axis=-1)
    for i in range(X.shape[-1]-1):
        prod_sin_alphas = tf.reduce_prod(tf.sin(alphas[..., :i]), axis=-1)
        updated_value = tf.acos(X_normd[..., i] / (prod_sin_alphas))
        indices = tf.constant([[i]])
        updates = tf.reshape(updated_value, [1])
        alphas = tf.tensor_scatter_nd_update(alphas, indices, updates)
    return alphas

In [7]:
# Define array and convert to parameters
array = tf.constant([1.0, 2.0, 3.0])
alphas = convert_array(array)
alphas

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.3002466 , 0.98279375], dtype=float32)>

In [8]:
def vector_loader(alphas, wires=None, is_x=True, is_conjugate=False):
    if wires is None:
        wires = list(range(len(alphas) + 1))
    if is_x and not is_conjugate:
        qml.PauliX(wires=wires[0])
    if is_conjugate:
        for i in range(len(wires) - 2, -1, -1):
            RBSGate(-alphas[i], wires=[wires[i], wires[i+1]])
    else:
        for i in range(len(wires) - 1):
            RBSGate(alphas[i], wires=[wires[i], wires[i+1]])
    if is_x and is_conjugate:
        qml.PauliX(wires=wires[0])

# Step 3: Create a quantum device
dev = qml.device('default.qubit', wires=4)

# Step 4: Construct the circuit using the custom gate
@qml.qnode(dev)
def pennylane_circuit():
    vector_loader(alphas.numpy(), is_x=True, is_conjugate=False)
    vector_loader(alphas.numpy(), is_x=True, is_conjugate=True)
    return qml.state()

In [9]:
# Execute the circuit and print the results -> expected |000>
state = pennylane_circuit()
print("State vector:")
print(state)

# Draw the circuit
print("Circuit diagram:")
print(qml.draw(pennylane_circuit)())

State vector:
[ 9.99999996e-01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
 -4.66223021e-09+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j]
Circuit diagram:
0: ──X─╭RBS(1.30)────────────────────────╭RBS(-1.30)──X─┤  State
1: ────╰RBS(1.30)─╭RBS(0.98)─╭RBS(-0.98)─╰RBS(-1.30)────┤  State
2: ───────────────╰RBS(0.98)─╰RBS(-0.98)────────────────┤  State


## Matrix Loader

In [10]:
def convert_matrix(X):
    mag_alphas = convert_array(tf.sqrt(tf.reduce_sum(X**2, axis=1)))
    alphas = tf.TensorArray(dtype=X.dtype, size=X.shape[0])
    for i in range(X.shape[0]):
        alphas = alphas.write(i, convert_array(X[i]))
    alphas = alphas.stack()
    return mag_alphas, alphas

# Define the array and convert to parameters
matrix = tf.constant([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])
mag_alphas, alphas = convert_matrix(matrix)
mag_alphas, alphas

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.3473037, 1.0086055], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[1.3002466 , 0.98279375],
        [1.0974779 , 0.87605804],
        [1.0442266 , 0.84415406]], dtype=float32)>)

In [11]:
def matrix_loader(mag_alphas, alphas, mag_wires, wires):
      qml.PauliX(wires=mag_wires[0])
      qml.PauliX(wires=wires[0])
      vector_loader(mag_alphas, wires=mag_wires, is_x=False)
      for i in range(len(mag_wires)):
          qml.CNOT(wires=[mag_wires[i], wires[0]])
          vector_loader(alphas[i], wires=wires, is_x=False)
          if i != len(mag_alphas):
              vector_loader(alphas[i+1], wires=wires, is_x=False, is_conjugate=True)

# Define the quantum device
dev = qml.device('default.qubit', wires=6)

# Define the QNode for the circuit
@qml.qnode(dev)
def pennylane_circuit():
    matrix_loader(mag_alphas.numpy(), alphas.numpy(), [0,1,2], [3,4,5])
    return qml.state()

# Execute the circuit and print the results
#state = pennylane_circuit()
#print("State vector:")
#print(state)

# Draw the circuit
print("Circuit diagram:")
print(qml.draw(pennylane_circuit)())


Circuit diagram:
0: ──X─╭RBS(1.35)────────────╭●────────────────────────────────────────────────────────────
1: ────╰RBS(1.35)─╭RBS(1.01)─│────────────────────────────────────────────────╭●───────────
2: ───────────────╰RBS(1.01)─│────────────────────────────────────────────────│────────────
3: ──X───────────────────────╰X─╭RBS(1.30)────────────────────────╭RBS(-1.10)─╰X─╭RBS(1.10)
4: ─────────────────────────────╰RBS(1.30)─╭RBS(0.98)─╭RBS(-0.88)─╰RBS(-1.10)────╰RBS(1.10)
5: ────────────────────────────────────────╰RBS(0.98)─╰RBS(-0.88)──────────────────────────

──────────────────────────────────────────────────────────────┤  State
──────────────────────────────────────────────────────────────┤  State
─────────────────────────────────────╭●───────────────────────┤  State
─────────────────────────╭RBS(-1.04)─╰X─╭RBS(1.04)────────────┤  State
──╭RBS(0.88)─╭RBS(-0.84)─╰RBS(-1.04)────╰RBS(1.04)─╭RBS(0.84)─┤  State
──╰RBS(0.88)─╰RBS(-0.84)───────────────────────────╰RBS(0.84)─┤  State


## Butterfly Circuit

In [12]:
def butterfly_circuit(parameters, wires=None):
    # If wires is None, use all qubits in the circuit
    if wires is None:
        length = qml.device.wires
    else:
        # If wires is not None, ensure it's a list of qubits
        length = len(wires)
    if length > 1:
        n = length // 2
        x = 0
        for i in range(n):
            RBSGate(parameters[x], wires=([wires[i], wires[i + n]]))
            x += 1
        butterfly_circuit(parameters[x: (len(parameters) // 2 + x // 2)], wires=wires[:n])
        butterfly_circuit(parameters[(len(parameters) // 2 + x // 2):], wires=wires[n:])

# Define the quantum device
dev = qml.device('default.qubit', wires=8)

# Example usage
parameters = np.array([x for x in range(12)])

@qml.qnode(dev)
def pennylane_circuit():
    butterfly_circuit(parameters, wires=range(8))
    return qml.state()

# Draw the circuit
print("Circuit diagram:")
print(qml.draw(pennylane_circuit)())


Circuit diagram:
0: ─╭RBS(0.00)───────────────────────╭RBS(4.00)────────────╭RBS(6.00)─────────────┤  State
1: ─│──────────╭RBS(1.00)────────────│──────────╭RBS(5.00)─╰RBS(6.00)─────────────┤  State
2: ─│──────────│──────────╭RBS(2.00)─╰RBS(4.00)─│──────────╭RBS(7.00)─────────────┤  State
3: ─│──────────│──────────│──────────╭RBS(3.00)─╰RBS(5.00)─╰RBS(7.00)─────────────┤  State
4: ─╰RBS(0.00)─│──────────│──────────│──────────╭RBS(8.00)────────────╭RBS(10.00)─┤  State
5: ────────────╰RBS(1.00)─│──────────│──────────│──────────╭RBS(9.00)─╰RBS(10.00)─┤  State
6: ───────────────────────╰RBS(2.00)─│──────────╰RBS(8.00)─│──────────╭RBS(11.00)─┤  State
7: ──────────────────────────────────╰RBS(3.00)────────────╰RBS(9.00)─╰RBS(11.00)─┤  State


In [13]:
def pyramid_circuit(parameters, wires=None):
    # If wires is None, use all qubits in the circuit
    if wires is None:
        length = len(qml.device.wires)
    else:
        # If wires is not None, ensure it's a list of qubits
        length = len(wires)

    k = 0

    for i in range(2 * length - 2):
        j = length - abs(length - 1 - i)

        if i % 2:
            for _ in range(j):
                if _ % 2 == 0 and k < len(parameters):
                    RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]]))
                    k += 1
        else:
            for _ in range(j):
                if _ % 2 and k < len(parameters):
                    RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]]))
                    k += 1

In [14]:
# Define the quantum device
dev = qml.device('default.qubit', wires=4)

# Example usage
parameters = np.array([x for x in range(6)])

@qml.qnode(dev)
def pennylane_circuit():
    pyramid_circuit(parameters, wires=range(4))
    return qml.state()

# Execute the circuit and print the results
#state = pennylane_circuit()
#print("State vector:")
#aprint(state)

# Draw the circuit
print("Circuit diagram:")
print(qml.draw(pennylane_circuit)())

Circuit diagram:
0: ─╭RBS(0.00)────────────╭RBS(2.00)────────────╭RBS(5.00)─┤  State
1: ─╰RBS(0.00)─╭RBS(1.00)─╰RBS(2.00)─╭RBS(4.00)─╰RBS(5.00)─┤  State
2: ────────────╰RBS(1.00)─╭RBS(3.00)─╰RBS(4.00)────────────┤  State
3: ───────────────────────╰RBS(3.00)───────────────────────┤  State
